Trabalho 1

In [6]:
from metodos import *

import numpy as np
import matplotlib.pyplot as plt

$\dot C = -e^{-\frac{10}{T+273}}*C$ 

$\dot T = 1000 * e^{-\frac{10}{T+273}}*C - 10 * (T-20)$ 

Equação da concentração:

In [9]:
def dC_dt(c, t):
    return -np.exp((-10)/(t+273))*c

Equação da temperatura:

In [10]:
def dT_dt(c, t):
    return (1000*np.exp((-10)/(t+273))*c)-(10*(t-20))

Condições iniciais:

In [11]:
tini = ... # temperatura inicial em graus Celsius
cini = ... # concentração inicial em gmol/L

tempo = np.linspace(0, 10, 1000) # vetor de tempo de 0 a 10 segundos com 1000 pontos, ou seja, com um passo de 0.01 segundos

Obtem a solução:

In [ ]:
solucao_C, solucao_T = rk4(dC_dt, dT_dt, cini, tini, tempo) # vai ficar algo assim pra chamar a solução, a função rk4 está na pasta metodos

Plota a concentração:

In [ ]:
plt.plot(tempo, solucao_C, label='Concentração') # plota o gráfico da concentração
plt.xlabel('Tempo (s)')
plt.ylabel('Concentração (gmol/L)')
plt.legend()
plt.grid()
plt.show()

Plota a temperatura:

In [ ]:
plt.plot(tempo, solucao_T, label='Temperatura') # plota o gráfico da temperatura
plt.xlabel('Tempo (s)')
plt.ylabel('Concentração (gmol/L)')
plt.legend()
plt.grid()
plt.show()